# Learning rate schedules

## The problem

Some of you may have encountered the following problem: the loss goes up both on the validation data **and** the training data. For example:

```
epoch 0 - train loss: 419.63, validation loss: 286.75, validation acc: 81.33
epoch 1 - train loss: 281.37, validation loss: 254.80, validation acc: 83.29
epoch 2 - train loss: 259.46, validation loss: 244.19, validation acc: 83.71
epoch 3 - train loss: 249.15, validation loss: 236.96, validation acc: 84.28
epoch 4 - train loss: 244.40, validation loss: 236.74, validation acc: 84.38
epoch 5 - train loss: 237.71, validation loss: 232.80, validation acc: 84.49
epoch 6 - train loss: 234.99, validation loss: 230.82, validation acc: 84.76
epoch 7 - train loss: 232.98, validation loss: 226.97, validation acc: 85.13
epoch 8 - train loss: 229.36, validation loss: 230.22, validation acc: 84.75
epoch 9 - train loss: 229.14, validation loss: 228.68, validation acc: 84.80
epoch 10 - train loss: 228.05, validation loss: 225.27, validation acc: 85.34
epoch 11 - train loss: 227.48, validation loss: 225.92, validation acc: 85.13
epoch 12 - train loss: 227.46, validation loss: 225.61, validation acc: 84.95
epoch 13 - train loss: 228.14, validation loss: 227.56, validation acc: 85.06
epoch 14 - train loss: 228.73, validation loss: 227.38, validation acc: 84.83
```

Remember that we had a encountered a similar situation in one of the earlier in-class exercises? We were minimizing the function $y = x^2$ using Tensorflow. If the learning rate was too high, the optimization would diverge. Suppose that we started with $x = -20$. Given that the derivative is $2x$, optimization would diverge with a learning rate higher than *1.0*.

The same can happen in real-world neural network optimization: the optimization has found a 'narrow' local minimum and with a relatively high learning rate, we actually move out of the minimum rather than descending further!

Of course, we cannot just set a lower learning rate for the entire optimization, since a high learning rate is very effective during the first few epochs.

## The solution

The solution is using a so-called *learning rate schedule*. The idea is that you start out with a relatively high learning rate, which is reduced during training. Tensorflow supports a couple of schedules, but one of the most commonly used schedules is *exponential decay*. If $r'$ is the learning rate following the schedule, $r$ the initial learning rate, $d$ the decay rate, and $i$ the number of epochs, then:

$r' = rd^i$

Tensorflow permits a bit more fine-grained control over the decay on a sub-epoch level. Directly quoting the Tensorflow docs:

```
decayed_learning_rate = learning_rate *
                        decay_rate ^ (global_step / decay_steps)
```

The `global_step` is the number of times that the parameters were updated. If we set `decay_steps` to the number of batches in the training data, then we get (a smoothed version of) the equation above.

Implementation in Tensorflow is pretty easy. Suppose that we want to use a starting learning rate of *0.01*, a decay rate of *0.90*, and have 177 batches in our data set, then we could use exponential decay as follows:

```python
# A variable to keep track of how often the parameters were updated.
global_step = tf.Variable(0, trainable=False)

# Create operation that computes the current learning rate.
learning_rate = tf.train.exponential_decay(start_lr, global_step, 177, 0.90)

# Hook up the learning rate with the optimizer. This goes in two directions:
# the optimizer updates `global_step` and uses `learning_rate`.
self._train_op = tf.train.AdamOptimizer(learning_rate).minimize(losses, global_step=global_step)

```

## The result

Afterwards, the optimization does not diverge anymore and gives considerably better results:

```
epoch 0 - train loss: 431.59, validation loss: 288.72, validation acc: 81.53
epoch 1 - train loss: 283.68, validation loss: 257.80, validation acc: 83.06
epoch 2 - train loss: 260.62, validation loss: 246.21, validation acc: 83.80
epoch 3 - train loss: 248.31, validation loss: 239.78, validation acc: 84.23
epoch 4 - train loss: 238.89, validation loss: 230.04, validation acc: 84.72
epoch 5 - train loss: 230.14, validation loss: 227.43, validation acc: 85.03
epoch 6 - train loss: 223.06, validation loss: 222.41, validation acc: 85.14
epoch 7 - train loss: 216.40, validation loss: 220.11, validation acc: 85.37
epoch 8 - train loss: 210.45, validation loss: 219.14, validation acc: 85.42
epoch 9 - train loss: 206.04, validation loss: 216.99, validation acc: 85.72
epoch 10 - train loss: 202.42, validation loss: 214.39, validation acc: 85.73
epoch 11 - train loss: 197.71, validation loss: 212.16, validation acc: 85.95
epoch 12 - train loss: 194.89, validation loss: 210.87, validation acc: 86.01
epoch 13 - train loss: 192.98, validation loss: 209.13, validation acc: 86.22
epoch 14 - train loss: 190.06, validation loss: 208.05, validation acc: 86.27
```